In [1]:
import boto3
import pandas as pd
from io import StringIO

In [5]:
s3 = boto3.client('s3')
bucket_name = 'my-bucket'
file_name = '201508_trip_data.csv'


sql_stmt = "SELECT * FROM s3object s WHERE s.\"Start Station\" = 'San Jose City Hall'"


# making the request
req = s3.select_object_content(
    Bucket=bucket_name,
    Key=file_name,
    ExpressionType='SQL',
    Expression=sql_stmt,
    InputSerialization = {'CSV': {'FileHeaderInfo': 'Use'}},
    OutputSerialization = {'CSV': {}},
)

records = []

# looping through the payload of the AWS EventStream and getting one or more Records and Stats
for event in req['Payload']:
    if 'Records' in event:
        records.append(event['Records']['Payload'])
    elif 'Stats' in event:
        stats = event['Stats']['Details']


# converting the byte strings to strings and then joining them together to form one large string
file_str = ''.join(r.decode('utf-8') for r in records)

# doing StringIO(file_str) so it looks like CSV file to pd.read_csv()
select_df = pd.read_csv(StringIO(file_str), usecols=[1,2,3], names=['Duration', 'Start Date', 'Start Station'])
print(len(select_df))
print(select_df.head())

832
   Duration       Start Date       Start Station
0       409  8/31/2015 23:10  San Jose City Hall
1      2915  8/30/2015 22:32  San Jose City Hall
2      2915  8/30/2015 22:31  San Jose City Hall
3       447  8/28/2015 15:31  San Jose City Hall
4       756  8/27/2015 13:06  San Jose City Hall


In [4]:
print (stats)

{'BytesScanned': 43012650, 'BytesProcessed': 43012650, 'BytesReturned': 94037}


In [6]:
# reading the entire file from S3
df = pd.read_csv('s3://my-bucket/201508_trip_data.csv', usecols=[1,2,3])

# filtering for stations marked as San Jose City Hall
filter = df['Start Station'] == 'San Jose City Hall'
print(len(df[filter]))

832


In [18]:
def select_s3_csv(s3_client, bucket_name, file_name, sql_stmt):
    """
    Parameters
    ----------
    s3_client : botocore.client.S3
    bucket_name : str, name of the S3 bucket
    file_name : str, name of the CSV file
    sql_stmt : str, sql statement

    Returns
    -------
    records : list of byte strings, contents of the CSV file minus column headers
    stats : dict, stat details from the EventStream

    """ 
    
    # making the request
    req = s3_client.select_object_content(
        Bucket=bucket_name,
        Key=file_name,
        ExpressionType='SQL',
        Expression=sql_stmt,
        InputSerialization = {'CSV': {'FileHeaderInfo': 'Use'}},
        OutputSerialization = {'CSV': {}},
    )
    
    records = []
    
    # looping through the payload of the AWS EventStream and getting one or more Records and Stats
    for event in req['Payload']:
        if 'Records' in event:
            records.append(event['Records']['Payload'])
        elif 'Stats' in event:
            stats = event['Stats']['Details']
            
    return records, stats